In [2]:
day_2 = {  'mwf': 'monday, wednesday, friday',
        'tth': 'tuesday, thursday',
        'mon-fri': 'monday, tuesday, wednesday, thursday, friday',
        'mon - fri':'monday, tuesday, wednesday, thursday, friday',
        'mon-sat': 'monday, tuesday, wednesday, thursday, friday, saturday',
        'm-f': 'monday, tuesday, wednesday, thursday, friday',
        'sat-sun': 'saturday, sunday',
        'tue-thu': 'tuesday, wednesday, thursday',
        'monday-friday':'monday, tuesday, wednesday, thursday, friday',
        'monday-thursday':'monday, tuesday, wednesday, thursday',
        'm-th' : 'monday, tuesday, wednesday, thursday'}
def generate_day_combinations():
    # Full day names and their abbreviations
    days = {
        'monday': ['mon', 'm'],
        'tuesday': ['tue', 'tu', 't'],
        'wednesday': ['wed', 'we', 'w'],
        'thursday': ['thu', 'th'],
        'friday': ['fri', 'fr', 'f'],
        'saturday': ['sat', 'sa', 's'],
        'sunday': ['sun', 'su']
    }

    # Function to get full day range
    def get_full_day_range(start_full, end_full):
        start_idx = list(days.keys()).index(start_full)
        end_idx = list(days.keys()).index(end_full)
        if start_idx == end_idx:
            return start_full
        return ', '.join(list(days.keys())[start_idx:end_idx + 1])

    # Create comprehensive mapping
    day_abbreviation_mapping = {}
    day_abbreviation_mapping.update(day_2)
    # Generate all possible day range combinations
    for start_full, start_abbrs in days.items():
        for end_full, end_abbrs in days.items():
            # Skip if start day comes after end day
            if list(days.keys()).index(start_full) > list(days.keys()).index(end_full):
                continue

            # Get the full day range
            full_day_range = get_full_day_range(start_full, end_full)

            # Add full day range
            day_abbreviation_mapping[f"{start_full}-{end_full}"] = full_day_range
            day_abbreviation_mapping[f"{start_full} - {end_full}"] = full_day_range
            day_abbreviation_mapping[f"{start_full} to {end_full}"] = full_day_range

            # Add combinations of abbreviations
            for start_abbr in start_abbrs:
                for end_abbr in end_abbrs:
                    combinations = [
                        f"{start_abbr}-{end_abbr}",
                        f"{start_abbr} - {end_abbr}",
                        f"{start_abbr} to {end_abbr}"
                    ]
                    for combo in combinations:
                        day_abbreviation_mapping[combo] = full_day_range

                    # Add combinations for mix of single and abbreviated names
                    if start_abbr != start_full:
                        day_abbreviation_mapping[f"{start_abbr}-{end_full}"] = full_day_range
                        day_abbreviation_mapping[f"{start_abbr} - {end_full}"] = full_day_range
                        day_abbreviation_mapping[f"{start_abbr} to {end_full}"] = full_day_range
                    if end_abbr != end_full:
                        day_abbreviation_mapping[f"{start_full}-{end_abbr}"] = full_day_range
                        day_abbreviation_mapping[f"{start_full} - {end_abbr}"] = full_day_range
                        day_abbreviation_mapping[f"{start_full} to {end_abbr}"] = full_day_range

    return day_abbreviation_mapping


# Generate the mapping
day_abbreviation_mapping = generate_day_combinations()

# Optional: Print or inspect the mapping
for key, value in day_abbreviation_mapping.items():
    print(f"{key}: {value}")


mwf: monday, wednesday, friday
tth: tuesday, thursday
mon-fri: monday, tuesday, wednesday, thursday, friday
mon - fri: monday, tuesday, wednesday, thursday, friday
mon-sat: monday, tuesday, wednesday, thursday, friday, saturday
m-f: monday, tuesday, wednesday, thursday, friday
sat-sun: saturday, sunday
tue-thu: tuesday, wednesday, thursday
monday-friday: monday, tuesday, wednesday, thursday, friday
monday-thursday: monday, tuesday, wednesday, thursday
m-th: monday, tuesday, wednesday, thursday
monday-monday: monday
monday - monday: monday
monday to monday: monday
mon-mon: monday
mon - mon: monday
mon to mon: monday
mon-monday: monday
mon - monday: monday
mon to monday: monday
monday-mon: monday
monday - mon: monday
monday to mon: monday
mon-m: monday
mon - m: monday
mon to m: monday
monday-m: monday
monday - m: monday
monday to m: monday
m-mon: monday
m - mon: monday
m to mon: monday
m-monday: monday
m - monday: monday
m to monday: monday
m-m: monday
m - m: monday
m to m: monday
monday

In [3]:
import pandas as pd
import re
import json

# Load the data from the specified Excel file and sheet
file_path = '/Users/rsiddiq2/Documents/FBCENC Test.xlsx'
sheet1_data = pd.ExcelFile(file_path).parse('Sheet1')

# Rename the columns for convenience (if necessary)
sheet1_data.rename(columns=lambda x: x.strip(), inplace=True)  # Removing any leading/trailing whitespace

# Update column names to match your actual file's columns
agency_no_column = 'Parent Agency No.'
agency_name_column = 'Agency_Name' if 'Agency_Name' in sheet1_data.columns else 'Agency Name'
delivery_info_column = 'Delivery Info.' if 'Delivery Info.' in sheet1_data.columns else 'Delivery Information'

# Function to normalize time format
def normalize_time(hour, minute, am_pm,is_end_time=False):
    # If minutes are not provided, set them to "00"
   
    if not minute:
        minute = "00"
    if am_pm in ['a', 'p']:
        am_pm = f"{am_pm}m"
    '''
    # If AM/PM is not provided, default to "AM" for start times and "PM" for end times
    if not am_pm:
        am_pm = "am"  # Assuming AM for start times
    
    if not is_end_time and int(hour) >= 12 and int(hour) <=6 and am_pm == "am":
        am_pm = "pm"
   
    if is_end_time and am_pm == "am" and int(hour) >= 1 : 
        am_pm = "pm"
    '''
    if not am_pm:
        if int(hour) ==12:
            am_pm="pm"
        elif int(hour) >6 and int(hour)<12:
            am_pm="am"
        else:
            am_pm = "pm"
    # Return the time in the format "HH:MM AM/PM"
    return f"{hour}:{minute} {am_pm}"
    
# Function to normalize the days and times
def normalize_days_and_times(text):
    # Convert text to lowercase for consistent matching
    text = text.lower()

    # Replace specific abbreviations like "MWF" or "Mon-Fri"
    '''
    day_abbreviation_mapping = {
        'mwf': 'monday, wednesday, friday',
        'tth': 'tuesday, thursday',
        'mon-fri': 'monday, tuesday, wednesday, thursday, friday',
        'mon - fri':'monday, tuesday, wednesday, thursday, friday',
        'mon-sat': 'monday, tuesday, wednesday, thursday, friday, saturday',
        'm-f': 'monday, tuesday, wednesday, thursday, friday',
        'sat-sun': 'saturday, sunday',
        'tue-thu': 'tuesday, wednesday, thursday',
        'monday-friday':'monday, tuesday, wednesday, thursday, friday',
        'monday-thursday':'monday, tuesday, wednesday, thursday',
        'm-th' : 'monday, tuesday, wednesday, thursday',

    }
    '''
    for abbr, days in day_abbreviation_mapping.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', days, text)

    # Replace individual day abbreviations with full names
    day_mapping = {
        'mon': 'monday', 'tue': 'tuesday', 'wed': 'wednesday', 'thu': 'thursday',
        'fri': 'friday', 'sat': 'saturday', 'sun': 'sunday',
        'm': 'monday', 'w': 'wednesday','t': 'tuesday', 'f': 'friday',
        'sa': 'saturday', 'su' : 'sunday','fr':'friday', 'th': 'thursday',
        'thur':'thursday',
        'thurs':'thursday','tues':'tuesday'
    }
    for short, full in day_mapping.items():
        text = re.sub(r'\b' + re.escape(short) + r'\b', full, text, flags=re.IGNORECASE)

    # Replace plural days like "Thursdays" with singular form "Thursday"
    text = re.sub(r'\b(.*?s)\b', lambda m: m.group(1)[:-1], text)
    
    # Normalize time ranges in the format "8-5 pm" or "8 am - 5 pm"
    text = re.sub(
        r'(\d{1,2}):(\d{2})(a|p)\s*-\s*(\d{1,2}):(\d{2})(a|p)',
        lambda m: f"{normalize_time(m.group(1), m.group(2), m.group(3))} - {normalize_time(m.group(4), m.group(5), m.group(6), is_end_time=True)}",
        text, flags=re.IGNORECASE
    )

    # Handle patterns like "6a-6p" (without minutes)
    text = re.sub(
        r'(\d{1,2})(a|p)\s*-\s*(\d{1,2})(a|p)',
        lambda m: f"{normalize_time(m.group(1), '00', m.group(2))} - {normalize_time(m.group(3), '00', m.group(4), is_end_time=True)}",
        text, flags=re.IGNORECASE
    )

    # Normalize time ranges like "8-5 pm" or "8 am - 5 pm"
    text = re.sub(
        r'(\d{1,2})(?::(\d{2}))?\s*(am|pm)?\s*-\s*(\d{1,2})(?::(\d{2}))?\s*(am|pm)?',
        lambda m: f"{normalize_time(m.group(1), m.group(2), m.group(3))} - {normalize_time(m.group(4), m.group(5), m.group(6), is_end_time=True)}",
        text, flags=re.IGNORECASE
    )
    return text

# Apply the normalization function to the 'Delivery Info.' column
sheet1_data['Normalized_Delivery_Info'] = sheet1_data[delivery_info_column].apply(normalize_days_and_times)

def standardize_delivery_info(normalized_text):
    standardized_info = []

    # Define patterns for days, weeks, and times
    #day_pattern = r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)"
    day_pattern = r"(monday|tuesday|wednesday|thursday|friday|saturday|sunday)"
    week_pattern = r"(\d+(?:st|nd|rd|th))\s*(?:week)?"
    time_pattern = r"(\d{1,2}:\d{2}\s*(?:AM|PM)?(?:\s*[-to]+\s*\d{1,2}:\d{2}\s*(?:AM|PM)?)?)"

    # Split the text into parts for analysis
    parts = re.split(r'[;,]', normalized_text)

    current_days = []
    current_weeks = []
    start_time = "Unknown"
    end_time = "Unknown"

    # Iterate over each part to identify days, weeks, and times in sequence
    for part in parts:
        part = part.strip()
        #print(f"Processing part: {part}")  # Debugging statement to see the current part being processed

        # Extract weeks if mentioned
        weeks = re.findall(week_pattern, part, re.IGNORECASE)
        if weeks:
            current_weeks = weeks
            
        # Extract days
        days = re.findall(day_pattern, part)
        if days:
            current_days.extend(days)

        # Extract times if present
        time_match = re.search(time_pattern, part, re.IGNORECASE)
        if time_match:
            time_str = time_match.group().strip()
            #print(standardized_info)
            # Handle time ranges
            if '-' in time_str or 'to' in time_str:
                start_time, end_time = map(str.strip, re.split(r'-|to', time_str, flags=re.IGNORECASE))
            else:
                start_time = time_str
                end_time = time_str
                
            # Assign the found times to all current days
            if current_days:
                
                for day in current_days:
                    standardized_info.append({
                        "Day": day,
                        "Opening_Hour": start_time,
                        "Closing_Hour": end_time,
                        "Week": current_weeks if current_weeks else ["1", "2", "3", "4"]
                    })
                
                # Reset the current_days and current_weeks since we have assigned the times
                current_days = []
                current_weeks = []
                start_time = "Unknown"
                end_time = "Unknown"

    # If there are remaining days without assigned times, add them with "Unknown"
    if current_days:
        for day in current_days:
            standardized_info.append({
                "Day": day,
                "Opening_Hour": "Unknown",
                "Closing_Hour": "Unknown",
                "Week": current_weeks if current_weeks else ["1", "2", "3", "4"]
            })
    
    return json.dumps(standardized_info, indent=2)

# Apply standardization to the 'Normalized_Delivery_Info' column
sheet1_data['Standardized_Delivery_Info'] = sheet1_data['Normalized_Delivery_Info'].apply(standardize_delivery_info)


# Print out the standardized delivery info for verification
#print(sheet1_data[['Parent Agency No.', 'Standardized_Delivery_Info']].head())



In [25]:
for n in (901,45,133,55,66,47):
    

    
    
    
    
    #print(sheet1_data["Standardized_Delivery_Info_Set_1"][n])
    
    
    print("Original Text:",sheet1_data['Delivery Info.'][n])
    print("Normalized Text:",sheet1_data['Normalized_Delivery_Info'][n])
    print(sheet1_data['Standardized_Delivery_Info'][n])

Original Text: Tuesdays 8:30-11am 
Normalized Text: tuesday 8:30 am - 11:00 am 
[
  {
    "Day": "tuesday",
    "Opening_Hour": "8:30 am",
    "Closing_Hour": "11:00 am",
    "Week": [
      "1",
      "2",
      "3",
      "4"
    ]
  }
]
Original Text: 24/7
Normalized Text: 24/7
[]
Original Text: DD 1st & 3rd Thurs 9:30am-1:30pm; pantry 2nd & 4th Wed 6-7pm
Normalized Text: dd 1st & 3rd thursday 9:30 am - 1:30 pm; pantry 2nd & 4th wednesday 6:00 pm - 7:00 pm
[
  {
    "Day": "thursday",
    "Opening_Hour": "9:30 am",
    "Closing_Hour": "1:30 pm",
    "Week": [
      "1st",
      "3rd"
    ]
  },
  {
    "Day": "wednesday",
    "Opening_Hour": "6:00 pm",
    "Closing_Hour": "7:00 pm",
    "Week": [
      "2nd",
      "4th"
    ]
  }
]
Original Text: Wednesdays 9am-1pm
Normalized Text: wednesday 9:00 am - 1:00 pm
[
  {
    "Day": "wednesday",
    "Opening_Hour": "9:00 am",
    "Closing_Hour": "1:00 pm",
    "Week": [
      "1",
      "2",
      "3",
      "4"
    ]
  }
]
Original Text:

In [13]:
import pandas as pd
import json

# Load the data from the specified Excel file and sheet
file_path = '/Users/rsiddiq2/Documents/FBCENC Test.xlsx'
#sheet1_data = pd.ExcelFile(file_path).parse('Sheet1')

# Function to process the JSON data in the 'Standardized_Delivery_Info' column
def process_json_in_column(df, json_column):
    new_rows = []

    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        try:
            # Load the JSON data from the column (assuming it's in string format)
            json_data = json.loads(row[json_column])
            
            # Loop through each day entry in the JSON data
            for entry in json_data:
                day = entry["Day"]
                opening_hour = entry["Opening_Hour"]
                closing_hour = entry["Closing_Hour"]
                weeks = entry["Week"]
                
                # Create a new row for each week for the given day
                for week in weeks:
                    # Create a new row with the original columns and the extracted day/week info
                    new_row = {
                        "Parent Agency No.": row["Parent Agency No."],
                        "No.": row["No."],
                        "Name": row["Name"],
                        "Parent Agency No..1": row["Parent Agency No..1"],
                        "Name 2": row["Name 2"],
                        "Address": row["Address"],
                        "City": row["City"],
                        "State": row["State"],
                        "ZIP Code": row["ZIP Code"],
                        "Day": day,
                        "Opening_Hour": opening_hour,
                        "Closing_Hour": closing_hour,
                        "Week": week
                    }
                    new_rows.append(new_row)

        except Exception as e:
            print(f"Error processing row {index}: {e}")
    
    # Create a new DataFrame with the new rows
    return pd.DataFrame(new_rows)

# Apply the function to process the JSON column
expanded_df = process_json_in_column(sheet1_data, 'Standardized_Delivery_Info')

# Now 'expanded_df' contains the new rows with the additional info
print(expanded_df.head())  # Preview the first few rows


  Parent Agency No.      No.                  Name Parent Agency No..1 Name 2  \
0             D1001  D1001FP  ACTS of Vance County               D1001    NaN   
1             D1001  D1001FP  ACTS of Vance County               D1001    NaN   
2             D1001  D1001FP  ACTS of Vance County               D1001    NaN   
3             D1001  D1001FP  ACTS of Vance County               D1001    NaN   
4             D1001  D1001FP  ACTS of Vance County               D1001    NaN   

            Address       City State ZIP Code      Day Opening_Hour  \
0  201 S William St  Henderson    NC    27536   monday      8:30 am   
1  201 S William St  Henderson    NC    27536   monday      8:30 am   
2  201 S William St  Henderson    NC    27536   monday      8:30 am   
3  201 S William St  Henderson    NC    27536   monday      8:30 am   
4  201 S William St  Henderson    NC    27536  tuesday      8:30 am   

  Closing_Hour Week  
0     11:30 am    1  
1     11:30 am    2  
2     11:30 am    3 

In [14]:
expanded_df.head(40)

,Parent Agency No.,No.,Name,Parent Agency No..1,Name 2,Address,City,State,ZIP Code,Day,Opening_Hour,Closing_Hour,Week
0,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,monday,8:30 am,11:30 am,1
1,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,monday,8:30 am,11:30 am,2
2,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,monday,8:30 am,11:30 am,3
3,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,monday,8:30 am,11:30 am,4
4,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,tuesday,8:30 am,11:30 am,1
5,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,tuesday,8:30 am,11:30 am,2
6,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,tuesday,8:30 am,11:30 am,3
7,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,tuesday,8:30 am,11:30 am,4
8,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,wednesday,8:30 am,11:30 am,1
9,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,wednesday,8:30 am,11:30 am,2


In [7]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

file_path = '/Users/rsiddiq2/Documents/FBCENC Test.xlsx'
sheet1_data = pd.ExcelFile(file_path).parse('Sheet1')

sheet1_data.rename(columns=lambda x: x.strip(), inplace=True)
agency_no_column = 'Parent Agency No.'
delivery_info_column = 'Delivery Info.'

# Prepare data for training
rows = sheet1_data[delivery_info_column].dropna().tolist()
labels = ['valid' if 'mon' in row.lower() or 'fri' in row.lower() else 'invalid' for row in rows]

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(rows[:150], labels[:150], test_size=0.2, random_state=42)

# Load pre-trained tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

# Create PyTorch datasets
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)

test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels)
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Define a simple model using BERT
class BERTClassifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(BERTClassifier, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask=attention_mask)
        return output.logits

# Initialize model, loss function, and optimizer
model = BERTClassifier(model_name, num_labels=2)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

# Training loop
for epoch in range(3):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

# Normalize time formatting

def normalize_time(hour, minute, am_pm,is_end_time=False):
    if not minute:
        minute = "00"
    if am_pm in ['a', 'p']:
        am_pm = f"{am_pm}m"
    if not am_pm:
        if int(hour) ==12:
            am_pm="pm"
        elif int(hour) >6 and int(hour)<12:
            am_pm="am"
        else:
            am_pm = "pm"
    return f"{hour}:{minute} {am_pm}"

# Normalize days and times

def normalize_days_and_times(text):
    text = text.lower()

    text = re.sub(
        r'(\d{1,2})(?::(\d{2}))?\s*(am|pm)?\s*-\s*(\d{1,2})(?::(\d{2}))?\s*(am|pm)?',
        lambda m: f"{normalize_time(m.group(1), m.group(2), m.group(3))} - {normalize_time(m.group(4), m.group(5), m.group(6), is_end_time=True)}",
        text, flags=re.IGNORECASE
    )

    # Use trained AI model to classify and validate
    encodings = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']
    outputs = model(input_ids, attention_mask)
    prediction = torch.argmax(outputs, dim=1)

    if prediction.item() == 1:
        text = f"Valid: {text}"
    else:
        text = f"Invalid: {text}"
    
    return text

sheet1_data['Normalized_Delivery_Info'] = sheet1_data[delivery_info_column].apply(normalize_days_and_times)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
sheet1_data

,Parent Agency No.,No.,Name,Parent Agency No..1,Name 2,Address,City,State,ZIP Code,FBC County Code,...,Monitoring within the last six month,FST Date,FST done in last six months,Monitoring Expiration Date,Food Safety Training Exp. Date,UNC Activity Status,Location Code,FBC Agency Category Code,Delivery Info.,Normalized_Delivery_Info
0,D1001,D1001FP,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,VANCE,...,No,2023-09-07,No,2019-08-01,2025-02-28,ACTIVE,D,FOODPANTRY,Mon-Fri 8:30-11:30am for food assistance; must...,Invalid: friday
1,D1001,D1001SK,ACTS of Vance County,D1001,NaN,201 S William St,Henderson,NC,27536,VANCE,...,No,2023-09-07,No,2020-08-04,2025-02-28,ACTIVE,D,SOUP KITCH,"Monday-Friday, 10:30-11:15am",Invalid: friday
2,D1002,D1002GH,Alliance Rehabilitative Care - Men,D1002,Halfway House,1020 County Home Rd,Henderson,NC,27536,VANCE,...,No,2026-08-22,Yes,2020-11-06,2028-02-13,ACTIVE,D,GROUP HOME,24/7,Invalid: 24/7
3,D1005,D1005FP,Alston Chapel United Holy,D1005,Church,1832 Alston Chapel Rd,Pittsboro,NC,27312,CHATHAM,...,No,2023-09-03,No,2020-11-18,2025-02-24,ACTIVE,D,FOODPANTRY,MWF 12:45-1:30pm; 3rd Wednesday (Food Bags); S...,Valid: wednesday
4,D1005,D1005TF,Alston Chapel United Holy,D1005,Church,1832 Alston Chapel Rd,Pittsboro,NC,27312,CHATHAM,...,No,2023-08-24,No,2020-11-20,2025-02-14,ACTIVE,D,TEFAP,MWF 12:45-1:30pm; 3rd Wednesday (Food Bags); S...,Valid: wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,W1144,W1144FP,CR Resource Center Inc.,W1144,NaN,107 Mt. Zion St,Lake Waccamaw,NC,28450,COLUMBUS,...,No,2023-03-01,No,2025-07-07,2024-08-22,ACTIVE,W,FOODPANTRY,Monday & Tuesday 1-3pm,Valid: monday & tuesday 1:00 pm - 3:00 pm
932,W1147,W1147FP,Liberty Baptist Church of Wilmington,W1147,NaN,7957 Market St,Wilmington,NC,28411,NEWHANOVER,...,NaN,2023-12-15,No,NaT,2025-06-07,ACTIVE,W,FOODPANTRY,1st and 3rd Mondays 1-3pm,Valid: 1st and 3rd mondays 1:00 pm - 3:00 pm
933,W1152,W1152DV,Domestic Violence Shelter & Services,W1152,NaN,Confidential,Wilmington,NC,28405,NEWHANOVER,...,NaN,2022-10-28,No,NaT,2024-04-20,ACTIVE,W,DV,Monday-Friday 9am-5pm,Valid: monday-friday 9:00 am - 5:00 pm
934,W1160,W1160DD,"The Healing Place of New Hanover County, Inc.",W1160,NaN,1000 Medical Center Drive,Wilmington,NC,28401,NEWHANOVER,...,NaN,2025-07-13,Yes,NaT,2027-01-04,ACTIVE,W,DIRECT DST,2nd and 4th Monday 3:30-5pm while supplies last,Invalid: 2nd and 4th monday 3:30 pm - 5:00 pm ...


In [8]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

file_path = '/Users/rsiddiq2/Documents/FBCENC Test.xlsx'
sheet1_data = pd.ExcelFile(file_path).parse('Sheet1')

sheet1_data.rename(columns=lambda x: x.strip(), inplace=True)
agency_no_column = 'Parent Agency No.'
delivery_info_column = 'Delivery Info.'

# Load pre-trained model and tokenizer for sequence-to-sequence tasks
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Prepare training data
rows = sheet1_data[delivery_info_column].dropna().tolist()
training_rows = rows[:150]

# Function to preprocess text for training

def preprocess_training_data(rows):
    training_data = []
    for text in rows:
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)  # Normalize spaces
        training_data.append(f"normalize: {text}")
    return training_data

# Generate training inputs
train_inputs = preprocess_training_data(training_rows)
train_labels = []
for row in training_rows:
    # Replace patterns to create labels for training
    label = re.sub(r'(\bmon\b|\bmon\b-day)', 'monday', row, flags=re.IGNORECASE)
    label = re.sub(r'(\btue\b|\btues\b)', 'tuesday', label, flags=re.IGNORECASE)
    label = re.sub(r'(\bwed\b|\bweds\b)', 'wednesday', label, flags=re.IGNORECASE)
    label = re.sub(r'(\bthu\b|\bthurs\b)', 'thursday', label, flags=re.IGNORECASE)
    label = re.sub(r'(\bfri\b)', 'friday', label, flags=re.IGNORECASE)
    label = re.sub(r'(\bsat\b)', 'saturday', label, flags=re.IGNORECASE)
    label = re.sub(r'(\bsun\b)', 'sunday', label, flags=re.IGNORECASE)
    label = re.sub(r'(\b\d{1,2}(?:[:.]\d{2})?\s*[ap]m\b)', lambda m: m.group().replace('.', ':'), label)
    train_labels.append(label)

# Tokenize inputs and labels
def tokenize_data(data):
    encodings = tokenizer(data, padding=True, truncation=True, return_tensors="pt", max_length=128)
    return encodings

train_encodings = tokenize_data(train_inputs)
label_encodings = tokenize_data(train_labels)

# Fine-tuning the model
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    model.train()
    optimizer.zero_grad()
    outputs = model(input_ids=train_encodings['input_ids'], attention_mask=train_encodings['attention_mask'], labels=label_encodings['input_ids'])
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Generate normalized text using the fine-tuned model
def generate_normalized_text(text):
    input_text = f"normalize: {text.lower()}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model.generate(inputs['input_ids'], max_length=128, num_return_sequences=1)
    normalized_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return normalized_text

# Apply AI-based dictionary generation
sheet1_data['Normalized_Delivery_Info'] = sheet1_data[delivery_info_column].apply(generate_normalized_text)

# Print sample results
print(sheet1_data[['Parent Agency No.', 'Normalized_Delivery_Info']].head())


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


  Parent Agency No.                           Normalized_Delivery_Info
0             D1001  normalize: mon-fri 8:30-11:30am for food assis...
1             D1001  normalize normal normalize: monday-day, 10:30-...
2             D1002                                          24/7 24/7
3             D1005  at 12:45-1:30 pm; sundays 12:30-1 pm; sundays ...
4             D1005                  12:45-1:30 pm; sundays 12:30-1 pm


In [10]:
sheet1_data[['Parent Agency No.', 'Normalized_Delivery_Info']].head(40)

,Parent Agency No.,Normalized_Delivery_Info
0,D1001,normalize: mon-fri 8:30-11:30am for food assis...
1,D1001,"normalize normal normalize: monday-day, 10:30-..."
2,D1002,24/7 24/7
3,D1005,at 12:45-1:30 pm; sundays 12:30-1 pm; sundays ...
4,D1005,12:45-1:30 pm; sundays 12:30-1 pm
5,D1006,normalize: 2nd and 4th wednesday from 12:00pm ...
6,D1008,-pm
7,D1008,normalize: mwf/9am-12pm
8,D1009,Normal normal: 2nd and 4th thursdays from 5-7pm
9,D1009,"Mon, wed, fri 3-6pm"
